### Classification

In [1]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train", trust_remote_code=True)
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

/home/hewliyang/courses/audio-processing/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import pipeline

classifier = pipeline(
  task="audio-classification",
  model="anton-l/xtreme_s_xlsr_300m_minds14"
)

/home/hewliyang/courses/audio-processing/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassificatio

In [3]:
example = minds[0]
classifier(example["audio"]["array"])

[{'score': 0.9625310301780701, 'label': 'pay_bill'},
 {'score': 0.02867273986339569, 'label': 'freeze'},
 {'score': 0.0033497940748929977, 'label': 'card_issues'},
 {'score': 0.0020058017689734697, 'label': 'abroad'},
 {'score': 0.0008484323625452816, 'label': 'high_value_payment'}]

In [9]:
# check the actual label
minds.features["intent_class"].int2str(example["intent_class"])

'pay_bill'

### ASR

In [10]:
asr = pipeline("automatic-speech-recognition")
example = minds[0]
asr(example["audio"]["array"])

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
config.json: 100%|██████████| 1.60k/1.60k [00:00<00:00, 6.84MB/s]
model.safetensors: 100%|██████████| 378M/378M [00:33<00:00, 11.4MB/s] 
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification m

{'text': 'I WOULD LIKE TO PAY MY ELECTRICITY BILL USING MY CAD CAN YOU PLEASE ASSIST'}

### TTS

In [12]:
tts = pipeline("text-to-speech", model="suno/bark-small")
text = "Ladybugs have had important roles in culture and religion, being associated with luck, love, fertility and prophecy. "
output = tts(text)

pytorch_model.bin: 100%|██████████| 1.68G/1.68G [02:30<00:00, 11.1MB/s]
/home/hewliyang/courses/audio-processing/.venv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
generation_config.json: 100%|██████████| 4.91k/4.91k [00:00<00:00, 16.4MB/s]
tokenizer_config.json: 100%|██████████| 353/353 [00:00<00:00, 1.71MB/s]
vocab.txt: 100%|██████████| 996k/996k [00:00<00:00, 2.53MB/s]
tokenizer.json: 100%|██████████| 2.92M/2.92M [00:00<00:00, 3.85MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 561kB/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-en

In [14]:
Audio(output["audio"], rate=output["sampling_rate"])